In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, IntegerType, DateType, TimestampType, FloatType 

catalog_name = 'ecommerce' 

In [0]:
df_bronze = spark.table(f'{catalog_name}.bronze.brz_brands')
df_bronze.show(10)

In [0]:
# cleaning brand_name removing leading and trailing spaces
df_silver = df_bronze.withColumn('brand_name', F.trim(F.col('brand_name')))
df_silver.show(5)

In [0]:
# cleaning brand_code removing special characters
df_silver = df_silver.withColumn('brand_code',F.regexp_replace(F.col('brand_code'),r'[^A-Za-z0-9]', ''))
df_silver.show(10)

In [0]:
# Identifying Unique Characters in category_code
df_silver.select(F.col('category_code')).distinct().show()

In [0]:
# Replacing anamolies in category_code
anamolies = {
    'GROCERY' : 'GRCY',
    'BOOKS' : 'BKS',
    'TOYS' : 'TOY' 
}

df_silver = df_silver.replace(anamolies, subset='category_code')
df_silver.select('category_code').distinct().show()

In [0]:
# Write raw data to the Silver layer (catalog: ecommerce, schema: silver, table: slv_brands)

df_silver.write.mode('delta') \
        .mode('overwrite') \
        .option('mergeSchema', 'true') \
        .saveAsTable(f'{catalog_name}.silver.slv_brands')
